In [1]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

In [2]:
train_loader = torch.utils.data.DataLoader(
    datasets.LFWPeople('data/train', transform=transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ]), download=True),
    batch_size=1, shuffle=True
)

  0%|          | 0/243346528 [00:00<?, ?it/s]

Extracting data/train/lfw-py/lfw-funneled.tgz to data/train/lfw-py


  0%|          | 0/94770 [00:00<?, ?it/s]

  0%|          | 0/94727 [00:00<?, ?it/s]

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=True)

model.eval()

epsilons = [0, .05, .1, .15, .2, .25, .3]

cpu


Downloading: "https://github.com/pytorch/vision/zipball/v0.6.0" to /root/.cache/torch/hub/v0.6.0.zip
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

In [4]:
def get_image():
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        return data, target


def get_prediction(image):
    pred = model(image)
    pred = pred.argmax(dim=1, keepdim=True)
    return pred


def get_grad(image, target):
    image.requires_grad = True
    output = model(image)
    loss = nn.CrossEntropyLoss()(output, target)
    model.zero_grad()
    loss.backward()
    data_grad = image.grad.data
    return data_grad


def FGSM_attack(image, epsilon, data_grad):
    # Collect the element-wise sign of the data gradient
    sign_data_grad = data_grad.sign()
    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = image + epsilon*sign_data_grad
    # Adding clipping to maintain [0,1] range
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    # Return the perturbed image
    return perturbed_image


def show_examples(examples):
    cnt = 0
    fig = plt.figure()
    for (init_pred, final_pred, ex) in examples:
        cnt += 1
        plt.subplot(1, len(examples), cnt)
        plt.xticks([], [])
        plt.yticks([], [])
        if final_pred == 0:
            lab = 'Adversarial'
        else:
            lab = 'Original'
        plt.xlabel("{} -> {}".format(init_pred, final_pred, lab))
        plt.imshow(ex, cmap="gray")
    plt.show()

In [5]:
def test(model, device, test_loader, epsilon):
    correct = 0
    adv_examples = []
    for data, target in test_loader:
        # data, target = data.to(device), target.to(device)
        data, target = data, target
        data.requires_grad = True
        output = model(data)
        init_pred = output.max(1, keepdim=True)[1]
        if init_pred.item() != target.item():
            continue
        loss = nn.CrossEntropyLoss()(output, target)
        model.zero_grad()
        loss.backward()
        data_grad = data.grad.data
        perturbed_data = FGSM_attack(data, epsilon, data_grad)
        # perturbed_data = PGD_attack(model, data, epsilon, data_grad)
        # perturbed_data = Carlini_and_wagner_attack(model, data, epsilon, data_grad)
        output = model(perturbed_data)
        final_pred = output.max(1, keepdim=True)[1]
        if final_pred.item() == target.item():
            correct += 1
        else:
            if len(adv_examples) < 5:
                adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
                adv_examples.append((init_pred.item(), final_pred.item(), adv_ex))
    final_acc = correct/float(len(test_loader))
    print("Epsilon: {}\tTest Accuracy = {} / {} = {}".format(epsilon, correct, len(test_loader), final_acc))
    return final_acc, adv_examples

In [6]:
accuracies = []
examples = []

In [ ]:
for epsilon in epsilons:
    acc, ex = test(model, device, train_loader, epsilon)
    accuracies.append(acc)
    examples.append(ex)

Epsilon: 0	Test Accuracy = 0 / 13233 = 0.0
Epsilon: 0.05	Test Accuracy = 0 / 13233 = 0.0
Epsilon: 0.1	Test Accuracy = 0 / 13233 = 0.0
Epsilon: 0.15	Test Accuracy = 0 / 13233 = 0.0


In [ ]:
show_examples(ex)
plt.figure(figsize=(5, 5))
plt.plot(epsilons, accuracies, "*-")
plt.yticks(np.arange(0, 1.1, step=0.1))
plt.xticks(np.arange(0, .35, step=0.05))
plt.title("Accuracy vs Epsilon")
plt.xlabel("Epsilon")
plt.ylabel("Accuracy")
plt.savefig('acc_vs_eps.png')
plt.show()